In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#config = tf.ConfigProto(allow_soft_placement = True)
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.33)
#config.gpu_options.allow_growth = True

In [2]:
mnist = input_data.read_data_sets('MNIST_DATA',one_hot=True)

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

In [4]:
#初始化权值
def weight_varible(shape):
    initial = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [5]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

In [6]:
#改变图片的格式[batch,heighe,width,channels]
x_image = tf.reshape(x,[-1,28,28,1])

In [7]:
#初始化第一个卷积层的权值和偏置
W_conv1 = weight_varible([5,5,1,32])
b_conv1 = bias_variable([32])

In [8]:
#卷积,池化操作
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
#初始化第二个卷积层的权值和偏置
W_conv2 = weight_varible([5,5,32,64])
b_conv2 = bias_variable([64])

In [10]:
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [11]:
h_pool2.get_shape().as_list()#卷积，池化后的shape

[None, 7, 7, 64]

In [12]:
#初始化一个全连接权重
W_fc1 = weight_varible([7*7*64,500])
b_fc1 = bias_variable([500])

In [13]:
#h_pool2变成一维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_pool2_flat.get_shape()

TensorShape([Dimension(None), Dimension(3136)])

In [14]:
#求第一个全连接的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

In [15]:
#dropout操作
keep_prob = tf.placeholder(tf.float32)
h_fc_drop = tf.nn.dropout(h_fc1,keep_prob=keep_prob)

In [16]:
#初始化第二个全连接层
W_fc2 = weight_varible([500,10])
b_fc2 = bias_variable([10])

In [17]:
#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc_drop,W_fc2) + b_fc2)

In [18]:
#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
#训练
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
#计算准确率
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)),tf.float32))


In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoach in range(11):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob:0.5})
            if batch % 100 == 0:

                accuracy = sess.run(acc,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})

                print('epoach' +str(epoach)+'-----'+str(batch) + ' acc '+ str(accuracy))  


epoach0-----0 acc 0.1235
epoach0-----100 acc 0.6974
epoach0-----200 acc 0.825
epoach0-----300 acc 0.9246
epoach0-----400 acc 0.9403
epoach0-----500 acc 0.9492
epoach1-----0 acc 0.9485
epoach1-----100 acc 0.9545
epoach1-----200 acc 0.9512
epoach1-----300 acc 0.9631
epoach1-----400 acc 0.9604
epoach1-----500 acc 0.9674
epoach2-----0 acc 0.9671
epoach2-----100 acc 0.9674
epoach2-----200 acc 0.9682
epoach2-----300 acc 0.9706
epoach2-----400 acc 0.972
epoach2-----500 acc 0.9722
epoach3-----0 acc 0.9756
epoach3-----100 acc 0.974
epoach3-----200 acc 0.9759
epoach3-----300 acc 0.9765
epoach3-----400 acc 0.9771
epoach3-----500 acc 0.9761
epoach4-----0 acc 0.9756
epoach4-----100 acc 0.9776
epoach4-----200 acc 0.978
epoach4-----300 acc 0.9776
epoach4-----400 acc 0.9779
epoach4-----500 acc 0.9787
epoach5-----0 acc 0.9798
epoach5-----100 acc 0.9802
epoach5-----200 acc 0.9814
epoach5-----300 acc 0.9805
epoach5-----400 acc 0.9813
epoach5-----500 acc 0.9813
epoach6-----0 acc 0.9824
epoach6-----100 acc